In [1]:
# all neccessary imports
import torch
import os
from PIL import Image
import label_converter # make sure the label_converter.py is in the folder with this notebook
import numpy as np

ModuleNotFoundError: No module named 'torch'

In [ ]:
# define paths
PATH_TO_IMAGES = "/Users/aleksandraivanova/Downloads/image"
PATH_TO_MODEL = "/Users/aleksandraivanova/Downloads/wetransfer_class_conversion-csv_2024-01-12_1104/model.pt"

In [6]:
# load model and print architecture
model = torch.load(PATH_TO_MODEL, map_location=torch.device('cpu'))
model.state_dict

In [9]:
# Convert .jpg to .tif
# Input and output folder paths
input_folder = PATH_TO_IMAGES
output_folder = PATH_TO_IMAGES

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# Loop through all files in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith(".jpg") or filename.endswith(".jpeg"):
        # Create the full path for the input and output files
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, os.path.splitext(filename)[0] + ".tif")

        # Open the JPG image
        image = Image.open(input_path)

        # Save the image in TIF format
        image.save(output_path, "TIFF")

        # Optionally, you can specify compression options for TIFF
        # image.save(output_path, "TIFF", compression="tiff_deflate")

In [36]:
converter = label_converter.LabelConverter()

def retrieve_label(filename):
        '''Define dataset-specific function, to retrieve true label for file.
        Here, Extracts data from the conversion table.                  '''
        
        true_lbl = str(filename[:2])
        try:
            result = converter[true_lbl]
            return result
        except KeyError:
            return -1

In [38]:
# Create dataset
root_dirs = [PATH_TO_IMAGES]
data = []
targets = []
for sgl_dir in root_dirs:
    print("Working on ", sgl_dir)
    for file_sgl in os.listdir(sgl_dir):
        print("Processing image ", file_sgl)

        if not '.tif' in file_sgl:
                    continue

        true_lbl = str(file_sgl[:2])
        converter.add(true_lbl, 1, split='test')

        label = retrieve_label(file_sgl)

        if not (true_lbl in converter.exclude):
                data.append(os.path.join(sgl_dir, file_sgl))
                targets.append(label)

Working on  /Users/aleksandraivanova/Downloads/image


In [40]:
print(data)
print(targets)

['/Users/aleksandraivanova/Downloads/image/LYT_1974.tif', '/Users/aleksandraivanova/Downloads/image/LYT_3805.tif', '/Users/aleksandraivanova/Downloads/image/NGS_2106.tif', '/Users/aleksandraivanova/Downloads/image/LYT_1021.tif', '/Users/aleksandraivanova/Downloads/image/MON_0220.tif', '/Users/aleksandraivanova/Downloads/image/EOS_0107.tif', '/Users/aleksandraivanova/Downloads/image/LYT_0303.tif', '/Users/aleksandraivanova/Downloads/image/NGS_1341.tif', '/Users/aleksandraivanova/Downloads/image/NGS_0739.tif', '/Users/aleksandraivanova/Downloads/image/EOS_0113.tif', '/Users/aleksandraivanova/Downloads/image/MON_0793.tif', '/Users/aleksandraivanova/Downloads/image/NGS_1802.tif', '/Users/aleksandraivanova/Downloads/image/LYT_2931.tif', '/Users/aleksandraivanova/Downloads/image/LYT_0868.tif', '/Users/aleksandraivanova/Downloads/image/NGS_2304.tif', '/Users/aleksandraivanova/Downloads/image/LYT_1223.tif', '/Users/aleksandraivanova/Downloads/image/LYT_0115.tif', '/Users/aleksandraivanova/Down

In [98]:
def get_image(idx):
        '''returns specific item from this dataset'''

        # load image, remove alpha channel, transform
        image = Image.open(data[idx])
        image_arr = np.asarray(image)[:,:,:3]
        image = Image.fromarray(image_arr)

        # load label
        label = targets[idx]

        return torch.tensor(image_arr), label

In [101]:
for param in model.parameters():
    print(param.dtype)

torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.

In [102]:
input = get_image(0)[0]
input = input.permute(2, 0, 1).unsqueeze(0)
# Convert the input tensor to the expected data type (Byte)
# ???
input = input.to(torch.uint8)

print(input.shape)
print(input.dtype)

model.eval()
pred = model(input)
print(pred)

torch.Size([1, 3, 400, 400])
torch.uint8


RuntimeError: expected scalar type Byte but found Float